In [1]:
# Implementation of HighEntropy and Random methods. Candidate adjective extraction is inclued in this notebook.

# Label
# 0: Physical
# 1: Mental

# Load Environment

In [80]:
'''
To use nltk's wordnet, you should download WordNet from https://www.nltk.org/nltk_data/ and then unzip wordnet.zip
under your home directory with path as /home_path/nltk_data/corpora/

'''
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
import pandas as pd
import numpy
import tqdm
import json
from collections import defaultdict
import joblib
import random

In [3]:
import nltk
"""
Number Tag Description
1.	CC	Coordinating conjunction
2.	CD	Cardinal number
3.	DT	Determiner
4.	EX	Existential there
5.	FW	Foreign word
6.	IN	Preposition or subordinating conjunction
7.	JJ	Adjective
8.	JJR	Adjective, comparative
9.	JJS	Adjective, superlative
10.	LS	List item marker
11.	MD	Modal
12.	NN	Noun, singular or mass
13.	NNS	Noun, plural
14.	NNP	Proper noun, singular
15.	NNPS	Proper noun, plural
16.	PDT	Predeterminer
17.	POS	Possessive ending
18.	PRP	Personal pronoun
19.	PRP$	Possessive pronoun
20.	RB	Adverb
21.	RBR	Adverb, comparative
22.	RBS	Adverb, superlative
23.	RP	Particle
24.	SYM	Symbol
25.	TO	to
26.	UH	Interjection
27.	VB	Verb, base form
28.	VBD	Verb, past tense
29.	VBG	Verb, gerund or present participle
30.	VBN	Verb, past participle
31.	VBP	Verb, non-3rd person singular present
32.	VBZ	Verb, 3rd person singular present
33.	WDT	Wh-determiner
34.	WP	Wh-pronoun
35.	WP$	Possessive wh-pronoun
36.	WRB	Wh-adverb
"""

# Usage Sample:
#text='The barbeque flavor of these chips is really perfect!'
#tokens = nltk.word_tokenize(text)
#tagged = nltk.pos_tag(tokens)


# self-defined tagging
def tagging(text,return_type='list'):
    tokens = nltk.word_tokenize(text)
    tagged = nltk.pos_tag(tokens)
    if return_type=='str':
        return " ".join(["%s /%s" % (w, tag) for w,tag in tagged])
    if return_type=='list':
        return [(w, tag) for w, tag in tagged]

tagging(text='The barbeque flavor of these chips is really perfect!')



[('The', 'DT'),
 ('barbeque', 'NN'),
 ('flavor', 'NN'),
 ('of', 'IN'),
 ('these', 'DT'),
 ('chips', 'NNS'),
 ('is', 'VBZ'),
 ('really', 'RB'),
 ('perfect', 'JJ'),
 ('!', '.')]

# Build (Adj, Noun) Relations


In [4]:
# extract "JJ+NN" relations by using pos_tags
def extract_relation(text,rtype='JJ+NN'):
    relations=[]
    pos_tags=tagging(text)
    
    if rtype=='JJ+NN':
        last_w=''
        last_tag=''
        for w,tag in pos_tags:
            if last_w:
                if 'J' in last_tag and 'N' in tag:
                    relations.append((last_w,w))
            last_w=w
            last_tag=tag
    
    return relations

# validate JJ+NN relations by WordNet POS flag
# if a word not exists in wordnet, we still keep this piece of relation. Further check will be done.
def validate_j_n(relation):
    adj, n =relation[0],relation[1]
    
    is_valid1=False
    syns=wn.synsets(n)
    if syns:
        for syn in syns:
            tag=syn.name().split('.')[1]
            if tag=='n':
                is_valid1=True
                break
    else: # if not exist in wordnet, not judge anything
        is_valid1=True
            
    is_valid2=False
    syns=wn.synsets(adj)
    if syns:
        for syn in syns:
            tag=syn.name().split('.')[1]
            if tag=='a' or tag == 's':
                is_valid2=True
                break
    else: # if not exist in wordnet, not judge anything
        is_valid2=True
        
    if is_valid1 and is_valid2:
        return True
    return False

validate_j_n(('extra','crunchy'))
    

True

In [11]:
# we don't provide this data as it's too large. 
# Download URL: https://www.kaggle.com/datasets/snap/amazon-fine-food-reviews

source_dir='amazon_food_review/'
nRowsRead = 1000000 # specify 'None' if want to read whole file
df = pd.read_csv(source_dir+'Reviews.csv', delimiter=',', nrows = nRowsRead)
df.dataframeName = 'Reviews.csv'
nRow, nCol = df.shape
print(f'There are {nRow} rows and {nCol} columns')

There are 568454 rows and 10 columns


In [12]:
relations=[]
for t in tqdm.tqdm(df['Text']):
    relations+=extract_relation(t,rtype='JJ+NN')

100%|██████████| 568454/568454 [34:25<00:00, 275.22it/s]  


In [13]:
joblib.dump(relations, 'Data/jj_nn_relation')

['Data/jj_nn_relation']

In [14]:
relations_new=[r for r in relations if validate_j_n(r)]

In [15]:
len(relations_new)

2497195

In [16]:
relations_new[:100]

[('several', 'of'),
 ('good', 'quality'),
 ('processed', 'meat'),
 ('better', 'than'),
 ('sure', 'if'),
 ('few', 'centuries'),
 ('pillowy', 'citrus'),
 ('tiny', 'squares'),
 ('powdered', 'sugar'),
 ('tiny', 'mouthful'),
 ('yummy', 'treat'),
 ('familiar', 'with'),
 ('secret', 'ingredient'),
 ('cherry', 'soda'),
 ('great', 'price'),
 ('wide', 'assortment'),
 ('wild', 'hair'),
 ('enjoyable', 'with'),
 ('many', 'flavors'),
 ('only', 'complaint'),
 ('much', 'red/black'),
 ('licorice-flavored', 'pieces'),
 ('particular', 'favorites'),
 ('delightful', 'treat'),
 ('great', 'flavors'),
 ('expensive', 'version'),
 ('beach-themed', 'party'),
 ('healthy', 'dog'),
 ('good', 'for'),
 ('small', 'puppies'),
 ('required', 'amount'),
 ('unique', 'combination'),
 ('hot', 'sauce'),
 ('bummed.', '<'),
 ('ecstatic', 'because'),
 ('it.', '<'),
 ('hot', 'sauce'),
 ('hot', 'sauce'),
 ('other', 'sauce.'),
 ('incredible', 'service'),
 ('skinny', 'boy'),
 ('higher', 'food'),
 ('more', 'than'),
 ('new', 'bag'),
 (

In [52]:
# check bad adj extracted directly from `relations`

limit=100
visited=set()
for r in tqdm.tqdm(relations):
    w=r[0]
    if w not in visited and ('non-' in w or '.' in w):
        print(w)
        visited.add(w)
        if len(visited)>limit:
            break

  0%|          | 3958/2828025 [00:00<00:02, 1015168.79it/s]

bummed.
it.
advertised.
non-coffee
issue.
non-instant
up.
great.
often.
convenient.
..
these.
problems.
tuned.
like.
up'.
lower.
refreshing.
lunch.
deal.There
go.Fifty
sugar.The
them.
skinny.
effective.
much.
oil.
all.
dogs.
safe.
ingredients.
out.
anything.
non-organic
first.
fans.
fluffy.
allergy.
non-baked
feet.
beer.
crispy.
exists.
saying.
adult.
about.
required.
there.
long.
opposite.
rancid.
now.
used.
stale.
again.
ok.
scissors.
too.
for.
non-existent.
natural.
non-manufactured
important.
'affection-ados'.
bergamot.
ready.
well.
right.
3.5-Ounce
treat.
flat.
widely.
2-pack.
tasty.
weight.
organic.
delicious.
us.
good.
.....
favorite.
bothersome.
non-animal
dishes.
ordered.
nice.
work.
unwind.
bigger.
alive.
dinner.
quickly.
squeaky.
differently.
straight.
perfect.
amazon.com-land
fast.
anymore.
eating.
non-healthful


# Extract Valid Adjectives 

    Actually we could scan each piece of review word by word and use WordNet to judge if a word is adjective. Instead, in this work we first build JJ+NN relations and then get adj from those relations. There should not be much difference between these two paths as the review corpus is extremely large with more than half a million samples. Moreover, we need this set of relations to build common knowledge about nouns.

In [5]:
relations=joblib.load('Data/jj_nn_relation')

In [23]:
# get all adjectives
adj_all=set([x[0] for x in relations_new])

In [24]:
len(adj_all)

46617

In [37]:
for syn in wn.synsets('healthier'):
    tag=syn.name().split('.')[1]
    if (tag=='a' or tag == 's'): # match
        print(syn.definition())

improved in health or physical condition
having or indicating good health in body or mind; free from infirmity or disease
financially secure and functioning well
promoting health; healthful; ; ; ; ; - C.B.Davis
exercising or showing good judgment
large in amount or extent or degree


In [19]:
# validate word type by WordNet
def wn_validate_type(word,word_type):
    res=[]
    syns=wn.synsets(word)
    for syn in syns:
        tag=syn.name().split('.')[1]
        if word_type=='adj' and (tag=='a' or tag == 's'): # match
            return True
        if word_type=='verb' and (tag=='v' or tag == 'v'): # match
            return True
    return False

# extract word definition from WordNet
def wn_ext_defn(word,word_type):
    res=[]
    syns=wn.synsets(word)
    for syn in syns:
        tag=syn.name().split('.')[1]
        if word_type=='adj' and (tag=='a' or tag == 's'): # match
            res.append(syn.definition())
        if word_type=='verb' and (tag=='v' or tag == 'v'): # match
            res.append(syn.definition())
    return '; '.join(res)
wn_ext_defn(word='burnt',word_type='adj')     

'ruined by overcooking; treated by heating to a high temperature but below the melting or fusing point; destroyed or badly damaged by fire'

In [26]:
# Validate word definitions provided by WordNet
# func `wn_validate_type` is actually not necessary whose function has already fulfilled in `validate_j_n`

adjset=[]
word_type='adj'
for w in tqdm.tqdm(set(adj_all)):
    w=w.replace('.','').replace("'",'')
    _defn=wn_ext_defn(w,word_type)
    if wn_validate_type(w,'adj') and _defn: # exclude []
        adjset.append((w,_defn,-1)) # -1: unknown

print(len(adjset))

adj=pd.DataFrame()
adj['word'],adj['text'],adj['target']=list(zip(*(adjset)))


100%|██████████| 46617/46617 [00:01<00:00, 43080.80it/s]


10939


In [78]:
# remove not-strings
# use lowercase
adj['word']=adj['word'].apply(lambda x:x.lower() if isinstance(x,str) else x)
adj[['word','text','target']].drop_duplicates().to_csv('Data/adj.csv')

In [6]:
# check size
adj=pd.read_csv('Data/adj.csv')
adj.shape

(7292, 4)

In [7]:
# check words containing token '-'
cnt=0
z=[]
for w in adj['word'].tolist():
    if isinstance(w,str):
        w=w.lower()
        if '-' in w:
            cnt+=1
            z.append(w)

In [8]:
cnt

642

In [9]:
random.shuffle(z)
z[:50]

['seventy-six',
 'stuck-up',
 'right-wing',
 'acid-tasting',
 'african-american',
 'true-blue',
 'off-the-shelf',
 'long-term',
 'twenty-four',
 'no-go',
 'tongue-in-cheek',
 'pumped-up',
 'oven-ready',
 'flesh-eating',
 'dust-covered',
 'fifty-four',
 'tailor-made',
 'make-believe',
 'half-hearted',
 'time-tested',
 'born-again',
 'first-year',
 'creamy-white',
 'high-voltage',
 'across-the-board',
 'tax-free',
 'full-bodied',
 'world-class',
 'hard-to-please',
 'mind-boggling',
 'twenty-one',
 'one-man',
 'self-propelled',
 'gray-green',
 'factory-made',
 'left-handed',
 'top-grade',
 'top-secret',
 'kidney-shaped',
 'sky-blue',
 'red-hot',
 'upside-down',
 'yellow-green',
 'strong-willed',
 'pancake-like',
 'off-line',
 'brand-new',
 'off-the-wall',
 'over-the-top',
 'ninety-five']

In [82]:
for w in adj['word'].tolist():
    if isinstance(w,str):
        w=w.lower()
        if 'non-' in w or '.' in w:
            print(w)

non-poisonous


In [83]:
wn.synsets('non-poisonous')[0].definition()

'not producing poison'

    From above exploration, there are 642/7292 adj with '-' inside and 1/7292 with 'non-'. All of them have valid definitions in WordNet. So we keep them as valid adjectives.

# Build TestSet

In [131]:
# Manually annotate a testset for evaluating model performance
# Randomly chosen from valid adjectives.
# Size of testset is 100.

import random
cand_words=list()
K=100
test_words=random.sample(adj['word'].tolist(),K)    
test_words_df=pd.DataFrame({'word':test_words})

testset=pd.merge(adj,test_words_df,on='word',how='inner')

# print out for annotation
_s=''
for w in test_words:
    _s+="'%s':0,"%w
#print(_s) 

In [67]:
# Annotation
# All labeling steps below are implemented by two annotators. If the labeling results disagree, 
# we ask a third expert to arbitrate.
test_man_labels={'teary-eyed':0,'meaningless':1,'41st':0,'unprotected':0,'malicious':1,'glossy':0,'discerning':1,
            'present':0,'federal':0,'noble':1,'air-dried':0,'together':1,'strong-willed':1,'circumspect':1,
            'developing':0,'unchanged':1,'impolite':1,'pavlovian':0,'limber':0,'braised':0,'obese':0,
            'carefree':1,'compositional':0,'temporary':0,'peppery':0,'granulated':0,'infinitesimal':0,
            'electrolytic':0,'lessened':1,'turbulent':0,'plain':0,'cute':1,'velvet':0,'unnameable':1,
            'singular':1,'infernal':1,'checked':0,'case-by-case':0,'skim':0,'engorged':0,'pro':1,'systemic':0,
            'done':1,'haphazard':0,'recurrent':0,'thinner':0,'galactic':0,'elapsed':0,'putrid':0,'noisiest':0,
            'disreputable':1,'arrant':0,'antsy':1,'corn-fed':0,'molecular':0,'genteel':1,'41':0,'hotter':0,
            'liver':0,'faster':0,'home-cured':0,'d':0,'nauseated':0,'relentless':1,'bouncy':0,'sensuous':1,
            'splendid':0,'stirring':1,'malleable':0,'permanent':0,'potent':0,'enamored':1,'laid':0,'gothic':0,
            'concerted':0,'unnamed':0,'finite':0,'caramel':0,'extra':1,'greenish':0,'incidental':1,'lazier':1,
            'medium-size':0,'sorry':1,'desperate':1,'nether':0,'algerian':0,'truncated':0,'pussy':0,
            'undisputed':1,'faddish':1,'tied':0,'fondest':1,'louder':0,'deadly':0,'automated':0,'manic':1,
            'delinquent':1,'emphasised':1,'rhetorical':1}
testset['target']=testset['word'].apply(lambda x: test_man_labels[x])
testset.to_csv('Data/test.csv')

In [21]:
# distribution 
cnt=0
for _w,_label in zip(testset['word'],testset['target']):
    if _label:
        cnt+=1
print(cnt/len(testset['word']))

0.36


# HighEntropy

    
    In each iteration, about 40 words are selected out for training. Iterations stop if performance converges.
    
    We run three times with different random seeds.

In [13]:
K=20 # number of positive samples in each round and negative samples have equal number.

## Iter1

### Setup

In [ ]:
adj.sample(frac=0.01)[['word','text']]

In [35]:
# Randomly choose some positive and negative words for training. Proportion of samples, 1:1
# `desc`: `physical` adj, describe physical attributes
# `opin`: `mental` adj, usually relates to mental attributes

desc=['anemic','arranged','assorted','available','baked','bitter','black','blue','broken','cherry',
      'citric','corrugated','commercial','cooked','crispy','crushed','crusty','decorative','dietetic',
     'digestible','dried','drippy','edible','empty','fake']
opin=['amazing','awesome','awful','aware','bold','bothersome','careful',
      'casual','certain','crazy','dulled','nosy','fussy','ignorant','preferable']

In [15]:
# If any word exists in testset, delete it and resample.
for w in desc+opin:
    if w in testset['word']:
        print(w)

In [20]:
word_type='adj'
trainset=[]
        
for w in desc:
    trainset.append((w,wn_ext_defn(w,word_type),0)) # 0: physical
for w in opin:
    trainset.append((w,wn_ext_defn(w,word_type),1)) # 1: mental


In [21]:
train=pd.DataFrame()
train['word'],train['text'],train['target']=list(zip(*(trainset)))

In [23]:
train.to_csv('Data/train.v6.1.round1.csv')

    Training is done by `train.py` with trainset `Data/train.v6.1.round1.csv`. Remember setting Config.is_predict=False in `train.py` before training. 

### Evaluate

    We use the trained model to make predictions over all valid adjectives. Higher the output probability, more 
    possible that this word is mental.
    
    The words with probability around 0.5 are uncertain words which need human-annotation for next round training.

In [22]:
pred=joblib.load('Data/pred.v6.1.1.round1') # load prediction result
assert pred.shape[0]==adj.shape[0]

In [11]:
# show uncertain words
uw=[]
for i,w in enumerate(adj['word'].tolist()):        
    if pred[i]>0.45 and pred[i]<0.55:
        uw.append((w,round(pred[i][0],2)))

In [12]:
len(uw)

4187

In [14]:
uw[:100]

[('adequate', 0.47),
 ('left-wing', 0.45),
 ('unconscionable', 0.47),
 ('pricier', 0.53),
 ('sublingual', 0.55),
 ('i', 0.46),
 ('orange-red', 0.52),
 ('darling', 0.49),
 ('ago', 0.46),
 ('folksy', 0.48),
 ('sloppy', 0.46),
 ('defective', 0.48),
 ('key', 0.46),
 ('judicious', 0.46),
 ('skeptical', 0.47),
 ('acceptable', 0.47),
 ('indiscernible', 0.48),
 ('empowered', 0.46),
 ('audible', 0.48),
 ('expedient', 0.46),
 ('eventual', 0.5),
 ('unsure', 0.48),
 ('newfound', 0.51),
 ('tippy', 0.46),
 ('converted', 0.46),
 ('oriental', 0.47),
 ('lamer', 0.48),
 ('artificial', 0.46),
 ('ill-fitting', 0.51),
 ('idle', 0.46),
 ('gastric', 0.45),
 ('stimulant', 0.46),
 ('humorous', 0.46),
 ('sixty-three', 0.52),
 ('accustomed', 0.48),
 ('behavioral', 0.45),
 ('wonderful', 0.48),
 ('likable', 0.47),
 ('croatian', 0.47),
 ('prepackaged', 0.46),
 ('overabundant', 0.49),
 ('overweight', 0.46),
 ('helpful', 0.47),
 ('unturned', 0.52),
 ('unsaturated', 0.45),
 ('unwarranted', 0.48),
 ('edged', 0.46),
 ('

## Iter2

### Setup

In [38]:
# choose round2 training samples from those most uncertain words in round1
desc2=['defective','indiscernible','empowered','audible','newfound','tippy','downward','wooly','oriental',
       'incendiary','prepackaged','wonderful','helpful','sealed','romantic','ongoing','adjacent','cheesy',
      'extra','present','warmer','noticeable','fifty-four',]

opin2=['humorous','sloppy','unnerved','judicious','burdensome','hip','acceptable','imbalanced','unsure',
       'accustomed','unnerved','angry','asserted','jerky','desired','depressed','dreamier','anguished',
      'unimpressed','unloved','infuriating','emphasised','homesick']


In [39]:
# If any word exists in testset, delete it and resample.
for w in desc2+opin2:
    if w in testset['word']:
        print(w)

In [40]:
len(desc2+opin2)

46

In [41]:
desc_all=desc+desc2
opin_all=opin+opin2


word_type='adj'
trainset=[]
for w in desc_all:
    trainset.append((w,wn_ext_defn(w,word_type),0)) # 0: physical
for w in opin_all:
    trainset.append((w,wn_ext_defn(w,word_type),1)) # 1: mental
    
    
train2=pd.DataFrame()
train2['word'],train2['text'],train2['target']=list(zip(*(trainset)))

train2.to_csv('Data/train.v6.2.round2.csv')

In [42]:
train2.shape

(86, 3)

    Training is done by `train.py` with trainset `Data/train.v6.2.round2.csv`. Remember setting Config.is_predict=False in `train.py` before training. 

### Evaluate

In [26]:
pred=joblib.load('Data/pred.v6.2.1.round2') # load prediction result
assert pred.shape[0]==adj.shape[0]

In [49]:
# show uncertain words

uw=[]
for i,w in enumerate(adj['word'].tolist()):
        
    if pred[i]>0.49 and pred[i]<0.51:
        uw.append((w,round(pred[i][0],2)))

In [50]:
len(uw)

172

In [51]:
random.shuffle(uw)
uw

[('reductive', 0.51),
 ('crisper', 0.49),
 ('habit-forming', 0.49),
 ('noticed', 0.5),
 ('tidier', 0.49),
 ('virtual', 0.5),
 ('discarded', 0.5),
 ('countless', 0.5),
 ('alluring', 0.51),
 ('recognizable', 0.51),
 ('haunted', 0.5),
 ('hourlong', 0.5),
 ('repeatable', 0.5),
 ('mythical', 0.5),
 ('controlled', 0.49),
 ('paler', 0.49),
 ('snazzy', 0.51),
 ('uppermost', 0.51),
 ('prime', 0.49),
 ('merciful', 0.5),
 ('upper', 0.5),
 ('enticing', 0.51),
 ('wacky', 0.5),
 ('full', 0.51),
 ('void', 0.49),
 ('uncontrollable', 0.5),
 ('tenable', 0.5),
 ('ambiguous', 0.51),
 ('whacky', 0.5),
 ('stuffy', 0.5),
 ('causal', 0.5),
 ('intricate', 0.5),
 ('riper', 0.5),
 ('fashionable', 0.5),
 ('restorative', 0.51),
 ('corrected', 0.5),
 ('liable', 0.5),
 ('shortest', 0.49),
 ('abysmal', 0.5),
 ('curly-haired', 0.5),
 ('lengthy', 0.5),
 ('evil', 0.49),
 ('strengthened', 0.5),
 ('glacial', 0.49),
 ('steeper', 0.5),
 ('orange-brown', 0.5),
 ('salvageable', 0.49),
 ('fullest', 0.51),
 ('cloudy', 0.51),
 (

## Iter3

### Setup

In [54]:
# Find more Mental Uncertain Words

uw=[]
for i,w in enumerate(adj['word'].tolist()):
        
    if pred[i]>0.51 and pred[i]<0.53:
        uw.append((w,round(pred[i][0],2)))
        
random.shuffle(uw)

for w,s in uw:
    t=adj.loc[adj.word==w,'text'].values[0]
    print("%s, %.3f, %s"%(w,s,t))
    print('--------')

repellent, 0.520, serving or tending to repel; highly offensive; arousing aversion or disgust; incapable of absorbing or mixing with
--------
stuffed, 0.520, filled with something; crammed with food
--------
humane, 0.520, pertaining to or concerned with the humanities; marked or motivated by concern with the alleviation of suffering; showing evidence of moral and intellectual advancement
--------
untested, 0.510, not tried or tested by experience; not yet proved or subjected to testing
--------
sanitized, 0.510, made sanitary
--------
pretentious, 0.510, making claim to or creating an appearance of (often undeserved) importance or distinction; intended to attract notice and impress others; (of a display) tawdry or vulgar
--------
funkier, 0.520, offensively malodorous; (of jazz) having the soulful feeling of early blues; stylish and modern in an unconventional way; in a state of cowardly fright
--------
unsightly, 0.530, unpleasant to look at
--------
educated, 0.530, possessing an ed

In [55]:
# annotate training samples from those most uncertain words in round2, 1:1
desc3=['reductive','crisper','noticed','discarded','countless','mythical','snazzy','uppermost','full',
       'tenable','ambiguous','stuffy','intricate','abysmal','strengthened','orange-brown','salvageable',
       'alive','stinking','nebulous']

opin3=['alluring','haunted','merciful','wacky','evil','conscionable','fiendish','addictive','demeaning',
       'appealing','ironic','understood','satisfactory','pretentious','repugnant','altruistic','kindest',
       'furious','horrific','fascinating']


In [56]:
# If any word exists in testset, delete it and resample.
for w in desc3+opin3:
    if w in testset['word']:
        print(w)

In [57]:
desc_all=desc+desc2+desc3
opin_all=opin+opin2+opin3


word_type='adj'
trainset=[]
for w in desc_all:
    trainset.append((w,wn_ext_defn(w,word_type),0)) # 0: physical
for w in opin_all:
    trainset.append((w,wn_ext_defn(w,word_type),1)) # 1: mental
    
    
train=pd.DataFrame()
train['word'],train['text'],train['target']=list(zip(*(trainset)))

train.to_csv('Data/train.v6.3.round3.csv')

In [58]:
train.shape

(126, 3)

    Training is done by `train.py` with trainset `Data/train.v6.3.round3.csv`. Remember setting Config.is_predict=False in `train.py` before training. 

### Evaluate

In [27]:
pred=joblib.load('Data/pred.v6.3.1.round3') # load prediction result
assert pred.shape[0]==adj.shape[0]

pred_dict={}
for i,w in enumerate(adj['word'].tolist()):
    pred_dict[w]=pred[i]

testset['pred_score']=testset['word'].apply(lambda x: pred_dict[x])
testset['pred_label']=testset['pred_score'].apply(lambda x: 1 if x>0.5 else 0)

# positive::precision
print("positive::precision %.3f"%(testset.loc[(testset.pred_label==1)&(testset.target==1)].shape[0]/testset.loc[(testset.pred_label==1)].shape[0]))
print("positive::recall %.3f"%(testset.loc[(testset.pred_label==1)&(testset.target==1)].shape[0]/testset.loc[(testset.target==1)].shape[0]))


print("negative::precision %.3f"%(testset.loc[(testset.pred_label==0)&(testset.target==0)].shape[0]/testset.loc[(testset.pred_label==0)].shape[0]))
print("negative::recall %.3f"%(testset.loc[(testset.pred_label==0)&(testset.target==0)].shape[0]/testset.loc[(testset.target==0)].shape[0]))

pp=testset.loc[(testset.pred_label==1)&(testset.target==1)].shape[0]/testset.loc[(testset.pred_label==1)].shape[0]
pr=testset.loc[(testset.pred_label==1)&(testset.target==1)].shape[0]/testset.loc[(testset.target==1)].shape[0]

print("F1: %.2f"%(2*pp*pr/(pp+pr)))

np=testset.loc[(testset.pred_label==0)&(testset.target==0)].shape[0]/testset.loc[(testset.pred_label==0)].shape[0]
nr=testset.loc[(testset.pred_label==0)&(testset.target==0)].shape[0]/testset.loc[(testset.target==0)].shape[0]

print("N F1: %.2f"%(2*np*nr/(np+nr)))

positive::precision 0.696
positive::recall 0.444
negative::precision 0.740
negative::recall 0.891
F1: 0.54
N F1: 0.81


### UncertainWords 

In [63]:
# Uncertain words
uw=[]
for i,w in enumerate(adj['word'].tolist()):        
    if pred[i]>0.48 and pred[i]<0.52:
        uw.append((w,round(pred[i][0],4)))

In [64]:
len(uw)

375

In [65]:
# show uncertain words
random.shuffle(uw)

for w,s in uw:
    t=adj.loc[adj.word==w,'text'].values[0]
    print("%s, %.3f, %s"%(w,s,t))
    print('--------')

involved, 0.483, connected by participation or association or use; entangled or hindered as if e.g. in mire; emotionally involved; highly complex or intricate and occasionally devious; ; ; ; ; ; ; ; - Sir Walter Scott; enveloped
--------
lenient, 0.492, tolerant or lenient; not strict; characterized by tolerance and mercy
--------
right-wing, 0.511, believing in or supporting tenets of the political right
--------
wispy, 0.495, thin and weak; - Edmund Wilson; lacking clarity or distinctness
--------
unchecked, 0.511, not restrained or controlled
--------
svelte, 0.494, showing a high degree of refinement and the assurance that comes from wide social experience; moving and bending with ease; being of delicate or slender build; - Frank Norris
--------
concerned, 0.480, feeling or showing worry or solicitude; involved in or affected by or having a claim to or share in; culpably involved
--------
offending, 0.517, offending against or breaking a law or rule
--------
barbecued, 0.519, cooke

transcendent, 0.484, exceeding or surpassing usual limits especially in excellence; beyond and outside the ordinary range of human experience or understanding
--------
classy, 0.513, elegant and fashionable; ; ; ; - Julia Child
--------
serious-minded, 0.498, acting with or showing thought and good sense
--------
gullible, 0.516, naive and easily deceived or tricked; easily tricked because of being too trusting
--------
unequaled, 0.508, radically distinctive and without equal
--------
wishful, 0.490, having or expressing desire for something; desiring or striving for recognition or advancement
--------
crazy, 0.505, affected with madness or insanity; foolish; totally unsound; possessed by inordinate excitement; bizarre or fantastic; intensely enthusiastic about or preoccupied with
--------
misguided, 0.480, poorly conceived or thought out; wrong in e.g. opinion or judgment
--------
innovative, 0.489, ahead of the times; being or producing something like nothing done or experienced or 

### RunTwoMore

    We only provide the evaluation results of another two runs here.

In [28]:
pred=joblib.load('Data/pred.v6.3.2.round3') # load prediction result
assert pred.shape[0]==adj.shape[0]

pred_dict={}
for i,w in enumerate(adj['word'].tolist()):
    pred_dict[w]=pred[i]


testset['pred_score']=testset['word'].apply(lambda x: pred_dict[x])
testset['pred_label']=testset['pred_score'].apply(lambda x: 1 if x>0.5 else 0)

# positive::precision
print("positive::precision %.3f"%(testset.loc[(testset.pred_label==1)&(testset.target==1)].shape[0]/testset.loc[(testset.pred_label==1)].shape[0]))
print("positive::recall %.3f"%(testset.loc[(testset.pred_label==1)&(testset.target==1)].shape[0]/testset.loc[(testset.target==1)].shape[0]))


print("negative::precision %.3f"%(testset.loc[(testset.pred_label==0)&(testset.target==0)].shape[0]/testset.loc[(testset.pred_label==0)].shape[0]))
print("negative::recall %.3f"%(testset.loc[(testset.pred_label==0)&(testset.target==0)].shape[0]/testset.loc[(testset.target==0)].shape[0]))

pp=testset.loc[(testset.pred_label==1)&(testset.target==1)].shape[0]/testset.loc[(testset.pred_label==1)].shape[0]
pr=testset.loc[(testset.pred_label==1)&(testset.target==1)].shape[0]/testset.loc[(testset.target==1)].shape[0]

print("F1: %.2f"%(2*pp*pr/(pp+pr)))

np=testset.loc[(testset.pred_label==0)&(testset.target==0)].shape[0]/testset.loc[(testset.pred_label==0)].shape[0]
nr=testset.loc[(testset.pred_label==0)&(testset.target==0)].shape[0]/testset.loc[(testset.target==0)].shape[0]

print("N F1: %.2f"%(2*np*nr/(np+nr)))

positive::precision 0.636
positive::recall 0.583
negative::precision 0.776
negative::recall 0.812
F1: 0.61
N F1: 0.79


In [29]:
pred=joblib.load('Data/pred.v6.3.4.round3') # load prediction result
assert pred.shape[0]==adj.shape[0]

pred_dict={}
for i,w in enumerate(adj['word'].tolist()):
    pred_dict[w]=pred[i]

testset['pred_score']=testset['word'].apply(lambda x: pred_dict[x])
testset['pred_label']=testset['pred_score'].apply(lambda x: 1 if x>0.5 else 0)

# positive::precision
print("positive::precision %.3f"%(testset.loc[(testset.pred_label==1)&(testset.target==1)].shape[0]/testset.loc[(testset.pred_label==1)].shape[0]))
print("positive::recall %.3f"%(testset.loc[(testset.pred_label==1)&(testset.target==1)].shape[0]/testset.loc[(testset.target==1)].shape[0]))


print("negative::precision %.3f"%(testset.loc[(testset.pred_label==0)&(testset.target==0)].shape[0]/testset.loc[(testset.pred_label==0)].shape[0]))
print("negative::recall %.3f"%(testset.loc[(testset.pred_label==0)&(testset.target==0)].shape[0]/testset.loc[(testset.target==0)].shape[0]))

pp=testset.loc[(testset.pred_label==1)&(testset.target==1)].shape[0]/testset.loc[(testset.pred_label==1)].shape[0]
pr=testset.loc[(testset.pred_label==1)&(testset.target==1)].shape[0]/testset.loc[(testset.target==1)].shape[0]

print("F1: %.2f"%(2*pp*pr/(pp+pr)))

np=testset.loc[(testset.pred_label==0)&(testset.target==0)].shape[0]/testset.loc[(testset.pred_label==0)].shape[0]
nr=testset.loc[(testset.pred_label==0)&(testset.target==0)].shape[0]/testset.loc[(testset.target==0)].shape[0]

print("N F1: %.2f"%(2*np*nr/(np+nr)))

positive::precision 0.778
positive::recall 0.583
negative::precision 0.795
negative::recall 0.906
F1: 0.67
N F1: 0.85


## Iter4

### Setup

In [78]:
# annotate round4 training samples from most uncertain words in round3
desc4=['involved','wispy','unchecked','offending','barbecued','detrimental','grizzly','beautiful','plush',
       'provable','granted','meatless','inseparable','after-school','jittery','key','nonalcoholic','asthmatic'
       ,'fizzy','nearby','second-best',]

opin4=['lenient','right-wing','concerned','impromptu','dastardly','deceptive','obedient','sincerest','obsequious',
       'craziest','disconcerting','ascetic','bracing','forgetful','convinced','bold','unpalatable','happy',
       'dissenting','inviting','sane',]

In [79]:
# If any word exists in testset, delete it and resample.
for w in desc4+opin4:
    if w in testset['word']:
        print(w)

In [80]:


desc_all=desc+desc2+desc3+desc4
opin_all=opin+opin2+opin3+opin4


word_type='adj'
trainset=[]
for w in desc_all:
    trainset.append((w,wn_ext_defn(w,word_type),0)) # 0: physical
for w in opin_all:
    trainset.append((w,wn_ext_defn(w,word_type),1)) # 1: mental
    
    
train=pd.DataFrame()
train['word'],train['text'],train['target']=list(zip(*(trainset)))

train.to_csv('Data/train.v6.4.round4.csv')

In [81]:
train.shape

(168, 3)

    Training is done by `train.py` with trainset `Data/train.v6.4.round4.csv`. Remember setting Config.is_predict=False in `train.py` before training. 

### Evaluate

In [30]:
pred=joblib.load('Data/pred.v6.4.1.round4') # load prediction result
assert pred.shape[0]==adj.shape[0]

pred_dict={}
for i,w in enumerate(adj['word'].tolist()):
    pred_dict[w]=pred[i]

testset['pred_score']=testset['word'].apply(lambda x: pred_dict[x])
testset['pred_label']=testset['pred_score'].apply(lambda x: 1 if x>0.5 else 0)

# positive::precision
print("positive::precision %.3f"%(testset.loc[(testset.pred_label==1)&(testset.target==1)].shape[0]/testset.loc[(testset.pred_label==1)].shape[0]))
print("positive::recall %.3f"%(testset.loc[(testset.pred_label==1)&(testset.target==1)].shape[0]/testset.loc[(testset.target==1)].shape[0]))


print("negative::precision %.3f"%(testset.loc[(testset.pred_label==0)&(testset.target==0)].shape[0]/testset.loc[(testset.pred_label==0)].shape[0]))
print("negative::recall %.3f"%(testset.loc[(testset.pred_label==0)&(testset.target==0)].shape[0]/testset.loc[(testset.target==0)].shape[0]))

pp=testset.loc[(testset.pred_label==1)&(testset.target==1)].shape[0]/testset.loc[(testset.pred_label==1)].shape[0]
pr=testset.loc[(testset.pred_label==1)&(testset.target==1)].shape[0]/testset.loc[(testset.target==1)].shape[0]

print("F1: %.2f"%(2*pp*pr/(pp+pr)))


np=testset.loc[(testset.pred_label==0)&(testset.target==0)].shape[0]/testset.loc[(testset.pred_label==0)].shape[0]
nr=testset.loc[(testset.pred_label==0)&(testset.target==0)].shape[0]/testset.loc[(testset.target==0)].shape[0]

print("N F1: %.2f"%(2*np*nr/(np+nr)))

positive::precision 0.808
positive::recall 0.583
negative::precision 0.797
negative::recall 0.922
F1: 0.68
N F1: 0.86


In [37]:
# show distribution
n,p=numpy.histogram(pred.flatten(),bins=20)
for i in range(n.shape[0]):
    print("[%.3f,%.3f]: %d"%(p[i],p[i+1],n[i]))

[0.112,0.145]: 1027
[0.145,0.179]: 1719
[0.179,0.213]: 851
[0.213,0.246]: 448
[0.246,0.280]: 278
[0.280,0.313]: 195
[0.313,0.347]: 173
[0.347,0.380]: 148
[0.380,0.414]: 136
[0.414,0.447]: 123
[0.447,0.481]: 116
[0.481,0.514]: 151
[0.514,0.548]: 134
[0.548,0.581]: 146
[0.581,0.615]: 162
[0.615,0.648]: 232
[0.648,0.682]: 304
[0.682,0.715]: 437
[0.715,0.749]: 381
[0.749,0.782]: 131


In [38]:
sum(pred>0.5)/pred.shape[0]

array([0.27550741])

### Uncertain Words

In [86]:
# show uncertain words
uw=[]
for i,w in enumerate(adj['word'].tolist()):        
    if pred[i]>0.48 and pred[i]<0.52:
        uw.append((w,round(pred[i][0],4)))
        
random.shuffle(uw)

for w,s in uw:
    t=adj.loc[adj.word==w,'text'].values[0]
    print("%s, %.3f, %s"%(w,s,t))
    print('--------')

smarter, 0.508, showing mental alertness and calculation and resourcefulness; elegant and stylish; characterized by quickness and ease in learning; improperly forward or bold; painfully severe; quick and brisk; capable of independent and apparently intelligent action
--------
utopian, 0.516, of or pertaining to or resembling a utopia; characterized by or aspiring to impracticable perfection
--------
all-night, 0.519, lasting, open, or operating through the whole night
--------
fluffier, 0.520, like down or as soft as down
--------
corny, 0.487, dull and tiresome but with pretensions of significance or originality
--------
monotonous, 0.507, tediously repetitious or lacking in variety; sounded or spoken in a tone unvarying in pitch
--------
competent, 0.491, properly or sufficiently qualified or capable or efficient; adequate for the purpose; legally qualified or sufficient
--------
freaky, 0.499, strange and somewhat frightening; conspicuously or grossly unconventional or unusual
-----

In [87]:
len(uw)

181

### RunTwoMore

    We only provide the evaluation results of another two runs here.

In [39]:
pred=joblib.load('Data/pred.v6.4.2.round4') # load prediction result
assert pred.shape[0]==adj.shape[0]

pred_dict={}
for i,w in enumerate(adj['word'].tolist()):
    pred_dict[w]=pred[i]


testset['pred_score']=testset['word'].apply(lambda x: pred_dict[x])
testset['pred_label']=testset['pred_score'].apply(lambda x: 1 if x>0.5 else 0)

# positive::precision
print("positive::precision %.3f"%(testset.loc[(testset.pred_label==1)&(testset.target==1)].shape[0]/testset.loc[(testset.pred_label==1)].shape[0]))
print("positive::recall %.3f"%(testset.loc[(testset.pred_label==1)&(testset.target==1)].shape[0]/testset.loc[(testset.target==1)].shape[0]))


print("negative::precision %.3f"%(testset.loc[(testset.pred_label==0)&(testset.target==0)].shape[0]/testset.loc[(testset.pred_label==0)].shape[0]))
print("negative::recall %.3f"%(testset.loc[(testset.pred_label==0)&(testset.target==0)].shape[0]/testset.loc[(testset.target==0)].shape[0]))

pp=testset.loc[(testset.pred_label==1)&(testset.target==1)].shape[0]/testset.loc[(testset.pred_label==1)].shape[0]
pr=testset.loc[(testset.pred_label==1)&(testset.target==1)].shape[0]/testset.loc[(testset.target==1)].shape[0]

print("F1: %.2f"%(2*pp*pr/(pp+pr)))


np=testset.loc[(testset.pred_label==0)&(testset.target==0)].shape[0]/testset.loc[(testset.pred_label==0)].shape[0]
nr=testset.loc[(testset.pred_label==0)&(testset.target==0)].shape[0]/testset.loc[(testset.target==0)].shape[0]

print("N F1: %.2f"%(2*np*nr/(np+nr)))

positive::precision 0.889
positive::recall 0.667
negative::precision 0.836
negative::recall 0.953
F1: 0.76
N F1: 0.89


In [41]:
# run another two times

pred=joblib.load('Data/pred.v6.4.3.round4') # load prediction result
assert pred.shape[0]==adj.shape[0]

pred_dict={}
for i,w in enumerate(adj['word'].tolist()):
    pred_dict[w]=pred[i]

testset['pred_score']=testset['word'].apply(lambda x: pred_dict[x])
testset['pred_label']=testset['pred_score'].apply(lambda x: 1 if x>0.5 else 0)

# positive::precision
print("positive::precision %.3f"%(testset.loc[(testset.pred_label==1)&(testset.target==1)].shape[0]/testset.loc[(testset.pred_label==1)].shape[0]))
print("positive::recall %.3f"%(testset.loc[(testset.pred_label==1)&(testset.target==1)].shape[0]/testset.loc[(testset.target==1)].shape[0]))


print("negative::precision %.3f"%(testset.loc[(testset.pred_label==0)&(testset.target==0)].shape[0]/testset.loc[(testset.pred_label==0)].shape[0]))
print("negative::recall %.3f"%(testset.loc[(testset.pred_label==0)&(testset.target==0)].shape[0]/testset.loc[(testset.target==0)].shape[0]))

pp=testset.loc[(testset.pred_label==1)&(testset.target==1)].shape[0]/testset.loc[(testset.pred_label==1)].shape[0]
pr=testset.loc[(testset.pred_label==1)&(testset.target==1)].shape[0]/testset.loc[(testset.target==1)].shape[0]

print("F1: %.2f"%(2*pp*pr/(pp+pr)))


np=testset.loc[(testset.pred_label==0)&(testset.target==0)].shape[0]/testset.loc[(testset.pred_label==0)].shape[0]
nr=testset.loc[(testset.pred_label==0)&(testset.target==0)].shape[0]/testset.loc[(testset.target==0)].shape[0]

print("N F1: %.2f"%(2*np*nr/(np+nr)))

positive::precision 0.735
positive::recall 0.694
negative::precision 0.833
negative::recall 0.859
F1: 0.71
N F1: 0.85


## Iter5

### Setup

In [90]:
# annotate round5 training-samples from most uncertain words in round4
desc5=['all-night','fluffier','monotonous','verbal','fictional','economic',
       'dressy','soapy','ago','ruined','unwholesome',
       'salutary','sweetish','overnight','bold','synthetic','dusky','starved']

opin5=['smarter','utopian','corny','freaky','prudish','neurotic','honored','fervent','decided','litigious',
       'liveliest','unholy','friendliest','irritable','bold','satisfying','smart','inviting','god-awful',
       'preferred',]

In [91]:
# If any word exists in testset, delete it and resample.
for w in desc5+opin5:
    if w in testset['word']:
        print(w)

In [94]:

desc_all=desc+desc2+desc3+desc4+desc5
opin_all=opin+opin2+opin3+opin4+opin5


word_type='adj'
trainset=[]
for w in desc_all:
    trainset.append((w,wn_ext_defn(w,word_type),0)) # 0: physical
for w in opin_all:
    trainset.append((w,wn_ext_defn(w,word_type),1)) # 1: mental
    
    
train=pd.DataFrame()
train['word'],train['text'],train['target']=list(zip(*(trainset)))

train.to_csv('Data/train.v6.5.round5.csv')

    Training is done by `train.py` with trainset `Data/train.v6.5.round5.csv`. Remember setting Config.is_predict=False in `train.py` before training. 

In [95]:
train.shape

(208, 3)

### Evaluate

In [42]:
pred=joblib.load('Data/pred.v6.5.6.round5') # load prediction result
assert pred.shape[0]==adj.shape[0]

pred_dict={}
for i,w in enumerate(adj['word'].tolist()):
    pred_dict[w]=pred[i]

testset['pred_score']=testset['word'].apply(lambda x: pred_dict[x])
testset['pred_label']=testset['pred_score'].apply(lambda x: 1 if x>0.5 else 0)

# positive::precision
print("positive::precision %.3f"%(testset.loc[(testset.pred_label==1)&(testset.target==1)].shape[0]/testset.loc[(testset.pred_label==1)].shape[0]))
print("positive::recall %.3f"%(testset.loc[(testset.pred_label==1)&(testset.target==1)].shape[0]/testset.loc[(testset.target==1)].shape[0]))


print("negative::precision %.3f"%(testset.loc[(testset.pred_label==0)&(testset.target==0)].shape[0]/testset.loc[(testset.pred_label==0)].shape[0]))
print("negative::recall %.3f"%(testset.loc[(testset.pred_label==0)&(testset.target==0)].shape[0]/testset.loc[(testset.target==0)].shape[0]))

pp=testset.loc[(testset.pred_label==1)&(testset.target==1)].shape[0]/testset.loc[(testset.pred_label==1)].shape[0]
pr=testset.loc[(testset.pred_label==1)&(testset.target==1)].shape[0]/testset.loc[(testset.target==1)].shape[0]

print("F1: %.2f"%(2*pp*pr/(pp+pr)))


np=testset.loc[(testset.pred_label==0)&(testset.target==0)].shape[0]/testset.loc[(testset.pred_label==0)].shape[0]
nr=testset.loc[(testset.pred_label==0)&(testset.target==0)].shape[0]/testset.loc[(testset.target==0)].shape[0]

print("N F1: %.2f"%(2*np*nr/(np+nr)))

positive::precision 0.781
positive::recall 0.694
negative::precision 0.838
negative::recall 0.891
F1: 0.74
N F1: 0.86


In [44]:
# show distribution
n,p=numpy.histogram(pred,bins=20)
for i in range(n.shape[0]):
    print("[%.3f,%.3f]: %d"%(p[i],p[i+1],n[i]))

[0.013,0.061]: 4020
[0.061,0.110]: 300
[0.110,0.158]: 150
[0.158,0.207]: 69
[0.207,0.255]: 69
[0.255,0.304]: 67
[0.304,0.352]: 62
[0.352,0.400]: 56
[0.400,0.449]: 37
[0.449,0.497]: 53
[0.497,0.546]: 54
[0.546,0.594]: 36
[0.594,0.642]: 52
[0.642,0.691]: 33
[0.691,0.739]: 52
[0.739,0.788]: 70
[0.788,0.836]: 73
[0.836,0.884]: 144
[0.884,0.933]: 257
[0.933,0.981]: 1638


In [45]:
sum(pred>0.5)/pred.shape[0]

array([0.33008777])

### UncertainWords

In [17]:
# Negative Uncertain words
uw=[]
for i,w in enumerate(adj['word'].tolist()):        
    if pred[i]>0.48 and pred[i]<0.5:
        uw.append((w,round(pred[i][0],4)))
random.shuffle(uw)

print(len(uw))

for w,s in uw[:K]:
    print(w)

147
verifiable
distinguished
replaceable
incontinent
erratic
four-year-old
five-year-old
andean
ball-shaped
flinty
unpalatable
myopic
blooded
anglo-indian
torturous
listed
pop
unisex
going
hot


### RunTwoMore

    We only provide the evaluation results of another two runs here.

In [46]:
pred=joblib.load('Data/pred.v6.5.8.round5') # load prediction result
assert pred.shape[0]==adj.shape[0]

pred_dict={}
for i,w in enumerate(adj['word'].tolist()):
    pred_dict[w]=pred[i]


testset['pred_score']=testset['word'].apply(lambda x: pred_dict[x])
testset['pred_label']=testset['pred_score'].apply(lambda x: 1 if x>0.5 else 0)

# positive::precision
print("positive::precision %.3f"%(testset.loc[(testset.pred_label==1)&(testset.target==1)].shape[0]/testset.loc[(testset.pred_label==1)].shape[0]))
print("positive::recall %.3f"%(testset.loc[(testset.pred_label==1)&(testset.target==1)].shape[0]/testset.loc[(testset.target==1)].shape[0]))


print("negative::precision %.3f"%(testset.loc[(testset.pred_label==0)&(testset.target==0)].shape[0]/testset.loc[(testset.pred_label==0)].shape[0]))
print("negative::recall %.3f"%(testset.loc[(testset.pred_label==0)&(testset.target==0)].shape[0]/testset.loc[(testset.target==0)].shape[0]))

pp=testset.loc[(testset.pred_label==1)&(testset.target==1)].shape[0]/testset.loc[(testset.pred_label==1)].shape[0]
pr=testset.loc[(testset.pred_label==1)&(testset.target==1)].shape[0]/testset.loc[(testset.target==1)].shape[0]

print("F1: %.2f"%(2*pp*pr/(pp+pr)))


np=testset.loc[(testset.pred_label==0)&(testset.target==0)].shape[0]/testset.loc[(testset.pred_label==0)].shape[0]
nr=testset.loc[(testset.pred_label==0)&(testset.target==0)].shape[0]/testset.loc[(testset.target==0)].shape[0]

print("N F1: %.2f"%(2*np*nr/(np+nr)))

positive::precision 0.826
positive::recall 0.528
negative::precision 0.779
negative::recall 0.938
F1: 0.64
N F1: 0.85


In [47]:
pred=joblib.load('Data/pred.v6.5.9.round5') # load prediction result
assert pred.shape[0]==adj.shape[0]

pred_dict={}
for i,w in enumerate(adj['word'].tolist()):
    pred_dict[w]=pred[i]


testset['pred_score']=testset['word'].apply(lambda x: pred_dict[x])
testset['pred_label']=testset['pred_score'].apply(lambda x: 1 if x>0.5 else 0)

# positive::precision
print("positive::precision %.3f"%(testset.loc[(testset.pred_label==1)&(testset.target==1)].shape[0]/testset.loc[(testset.pred_label==1)].shape[0]))
print("positive::recall %.3f"%(testset.loc[(testset.pred_label==1)&(testset.target==1)].shape[0]/testset.loc[(testset.target==1)].shape[0]))


print("negative::precision %.3f"%(testset.loc[(testset.pred_label==0)&(testset.target==0)].shape[0]/testset.loc[(testset.pred_label==0)].shape[0]))
print("negative::recall %.3f"%(testset.loc[(testset.pred_label==0)&(testset.target==0)].shape[0]/testset.loc[(testset.target==0)].shape[0]))

pp=testset.loc[(testset.pred_label==1)&(testset.target==1)].shape[0]/testset.loc[(testset.pred_label==1)].shape[0]
pr=testset.loc[(testset.pred_label==1)&(testset.target==1)].shape[0]/testset.loc[(testset.target==1)].shape[0]

print("F1: %.2f"%(2*pp*pr/(pp+pr)))


np=testset.loc[(testset.pred_label==0)&(testset.target==0)].shape[0]/testset.loc[(testset.pred_label==0)].shape[0]
nr=testset.loc[(testset.pred_label==0)&(testset.target==0)].shape[0]/testset.loc[(testset.target==0)].shape[0]

print("N F1: %.2f"%(2*np*nr/(np+nr)))

positive::precision 0.758
positive::recall 0.694
negative::precision 0.836
negative::recall 0.875
F1: 0.72
N F1: 0.85


    Annotate 60~70words/iter. Stop at iter5 as performance converges. Precision/Recall of both classes are already above 70%.

# Random

In [ ]:
# baseline
# randomly annotate words for training

In [42]:
adj_cand=[]
for w in adj['word']:
    if w not in testset['word']:
        adj_cand.append(w)

# randomly sampled 100 mentals and 100 physicals as training set
rs_trainset=random.sample(adj_cand,200)

In [43]:
for w in rs_trainset:
    print(w)

consuming
abnormal
splashier
dizzy
twilight
protected
incidental
plainer
medium
available
vascular
enteral
lamest
softened
crimson
black-haired
hit-or-miss
stupidest
fat-free
age-old
egyptian
radical
air-filled
doable
absurd
undeserving
petulant
comforting
stoned
zany
unresponsive
preferential
plastered
cardboard
low-level
qualitative
high-power
splitting
perfected
unobtrusive
farther
instinct
head-to-head
affected
disposable
brinier
wanton
vinegary
rising
gelatinous
bearable
nondescript
sicker
dissatisfied
high-tech
glace
outgoing
apocalyptic
void
mint
experienced
appreciative
blanket
29
starchy
silly
jealous
grown
sedimentary
third
long-life
axillary
semisolid
untruthful
middle
recovering
expandable
drastic
fervent
pet
loamy
96
chitinous
left-handed
chipper
attentive
powdery
slushy
mexican
unimproved
siamese
augmented
tailor-made
inadequate
lightheaded
tidal
obscure
legible
country-style
thai
latest
78
truncated
good-natured
hand-held
wanted
ritzy
unfeasible
freaky
extrinsic
drafty
p

In [256]:
# check definition text
adj.loc[adj.word=='shining','text'].values[0]

'marked by exceptional merit; made smooth and bright by or as if by rubbing; reflecting a sheen or glow; reflecting light'

In [78]:
# labeling
# `rs_desc`: `physical` adj, describe physical attributes
# `rs_opin`: `mental` adj, usually relates to mental attributes

rs_desc=['consuming','abnormal','splashier','twilight','protected','incidental','plainer','medium','available','vascular','enteral','lamest','softened','crimson','black-haired','fat-free','age-old','egyptian','air-filled','doable','preferential','plastered','cardboard','low-level','qualitative','high-power','splitting','perfected','farther','head-to-head','disposable','brinier','vinegary','rising','gelatinous','nondescript','sicker','high-tech','glace','apocalyptic','void','mint','blanket','29','starchy','grown','sedimentary','third','long-life','axillary','semisolid','middle','recovering','expandable','drastic','loamy','96','chitinous','left-handed','powdery','slushy','mexican','unimproved','siamese','augmented','tailor-made','inadequate','tidal','obscure','legible','country-style','thai','latest','78','truncated','hand-held','ritzy','unfeasible','extrinsic','drafty','premenstrual','squat','protean','deleterious','recurring','lit','fruitful','nitric','finite','prolific','unregulated','geographical','unrefined','slick','mushy','speediest','correct','fantastic','licensed','decreased','filthy','barbaric','copious','twenty-nine','vanished','sulfurous','greasiest','cookie-cutter','physical','110','amber','fifty-two','cultural','cupric','unsavory','resonant','arthritic','feline','personalized','muscle-bound','notched','fusty','rocky','popular','four-year-old','broader','later','external','unemployed','executive','induced','traceable','occidental','jagged','full-strength','precooked','japanese','enhanced','tenuous','helpless','tainted','windswept','several','slow','interstate','tried','visible','three-year-old','abandoned','psychoactive']
rs_opin=['dizzy','hit-or-miss','stupidest','radical','absurd','undeserving','petulant','comforting','stoned','zany','unresponsive','unobtrusive','instinct','affected','wanton','bearable','dissatisfied','outgoing','experienced','appreciative','silly','jealous','untruthful','fervent','pet','chipper','attentive','lightheaded','good-natured','wanted','freaky','boring','calmer','unintentional','sensitive','favorite','horrific','interesting','awfulest','heartsick','groggy','considered','euphemistic','communicative','heartier','dilatory','disgusting','canny','respectable','meek']

In [89]:
len(rs_desc) 

150

In [90]:
len(rs_opin) 

50

    As mental and physical words are distributed unbalancedly, we need to find more mentals for trainset.

In [92]:
# add 50 more mentals

rs_trainset2=random.sample(adj_cand,500)
for w in rs_trainset2:
    if w in rs_desc+rs_opin:
        continue
    else:
        print(w)

brown-haired
soft-boiled
monetary
historical
faulty
leisurely
sanitized
inconsiderable
mnemonic
wetter
fleshly
fewer
humic
lunatic
maniacal
indescribable
milkier
putative
all-purpose
ravenous
moderate-size
curricular
disabled
focused
sundried
21
falsest
rough-and-ready
incorporated
thoughtful
angry
bilious
smallish
honeyed
time-release
full-blown
varicose
material
anticlimactic
decayed
received
foul-smelling
counter
observed
speedy
folding
hair-raising
twisty
pasted
homemade
stirring
eccentric
brazilian
floral
continual
graduate
tamed
printable
angled
human-sized
fulfilled
convinced
senegalese
lacking
coveted
seedless
justifiable
second-rate
thawed
chock-full
frontal
under
hard-baked
wacky
acetic
deserving
spiffy
marvellous
unmeasured
missionary
fifty-eight
x
devoid
worldwide
indecisive
unwanted
thirsty
forgotten
pastier
petty
pituitary
sun-dried
essential
separable
expected
edwardian
roundish
rural
dressed
unstirred
stuck
stretchier
purer
plushy
ocular
rousing
extralegal
pointless
uns

In [100]:
rs_opin=rs_opin+['ravenous','focused','thoughtful','angry','stirring','wacky','expected','unsatisfiable','expectable','allusive','psychotic','lyrical','philosophical','impressed','striking','pessimistic','happy','autistic','adored','irritating','charismatic','crazier','restless','attractive','picky','devoted','enlightening','anticipated','nervy','prudent','discerning','ungenerous','preferred','gleeful','doubtful','shocked','worrisome','embarrassing','tolerant','hesitating','unwitting']

In [101]:
len(rs_opin)

91

In [102]:
# get 100 mentals and 100 physicals for training
word_type='adj'
rs_trainset=[]
for w in rs_desc[:100]:
    rs_trainset.append((w,wn_ext_defn(w,word_type),0)) # 0: physical
for w in rs_opin[:100]:
    rs_trainset.append((w,wn_ext_defn(w,word_type),1)) # 1: mental
    
    
rs_train=pd.DataFrame()
rs_train['word'],rs_train['text'],rs_train['target']=list(zip(*(rs_trainset)))

rs_train=rs_train.sample(frac=1).reset_index(drop=True)
rs_train.to_csv('Data/train.randomSample.csv')

In [103]:
rs_train.shape

(191, 3)

## Iter3

In [104]:
# use 120 words
rs_train=pd.read_csv('Data/train.randomSample.csv')

rs_train[:120].to_csv('Data/train.randomSample.r3.csv')

    Train model the same way as Section5.

In [49]:
# evaluate

pred=joblib.load('Data/pred.v5.12.round3') # load prediction result
assert pred.shape[0]==adj.shape[0]

pred_dict={}
for i,w in enumerate(adj['word'].tolist()):
    pred_dict[w]=pred[i]


testset['pred_score']=testset['word'].apply(lambda x: pred_dict[x])
testset['pred_label']=testset['pred_score'].apply(lambda x: 1 if x>0.5 else 0)

# positive::precision
print("positive::precision %.3f"%(testset.loc[(testset.pred_label==1)&(testset.target==1)].shape[0]/testset.loc[(testset.pred_label==1)].shape[0]))
print("positive::recall %.3f"%(testset.loc[(testset.pred_label==1)&(testset.target==1)].shape[0]/testset.loc[(testset.target==1)].shape[0]))


print("negative::precision %.3f"%(testset.loc[(testset.pred_label==0)&(testset.target==0)].shape[0]/testset.loc[(testset.pred_label==0)].shape[0]))
print("negative::recall %.3f"%(testset.loc[(testset.pred_label==0)&(testset.target==0)].shape[0]/testset.loc[(testset.target==0)].shape[0]))

pp=testset.loc[(testset.pred_label==1)&(testset.target==1)].shape[0]/testset.loc[(testset.pred_label==1)].shape[0]
pr=testset.loc[(testset.pred_label==1)&(testset.target==1)].shape[0]/testset.loc[(testset.target==1)].shape[0]

print("F1: %.2f"%(2*pp*pr/(pp+pr)))

np=testset.loc[(testset.pred_label==0)&(testset.target==0)].shape[0]/testset.loc[(testset.pred_label==0)].shape[0]
nr=testset.loc[(testset.pred_label==0)&(testset.target==0)].shape[0]/testset.loc[(testset.target==0)].shape[0]

print("N F1: %.2f"%(2*np*nr/(np+nr)))

positive::precision 0.658
positive::recall 0.694
negative::precision 0.823
negative::recall 0.797
F1: 0.68
N F1: 0.81


In [50]:
# another two runs. Only provide evaluations here.

pred=joblib.load('Data/pred.v5.1.3.round3') # load prediction result
assert pred.shape[0]==adj.shape[0]

pred_dict={}
for i,w in enumerate(adj['word'].tolist()):
    pred_dict[w]=pred[i]


testset['pred_score']=testset['word'].apply(lambda x: pred_dict[x])
testset['pred_label']=testset['pred_score'].apply(lambda x: 1 if x>0.5 else 0)

# positive::precision
print("positive::precision %.3f"%(testset.loc[(testset.pred_label==1)&(testset.target==1)].shape[0]/testset.loc[(testset.pred_label==1)].shape[0]))
print("positive::recall %.3f"%(testset.loc[(testset.pred_label==1)&(testset.target==1)].shape[0]/testset.loc[(testset.target==1)].shape[0]))


print("negative::precision %.3f"%(testset.loc[(testset.pred_label==0)&(testset.target==0)].shape[0]/testset.loc[(testset.pred_label==0)].shape[0]))
print("negative::recall %.3f"%(testset.loc[(testset.pred_label==0)&(testset.target==0)].shape[0]/testset.loc[(testset.target==0)].shape[0]))

pp=testset.loc[(testset.pred_label==1)&(testset.target==1)].shape[0]/testset.loc[(testset.pred_label==1)].shape[0]
pr=testset.loc[(testset.pred_label==1)&(testset.target==1)].shape[0]/testset.loc[(testset.target==1)].shape[0]

print("F1: %.2f"%(2*pp*pr/(pp+pr)))

np=testset.loc[(testset.pred_label==0)&(testset.target==0)].shape[0]/testset.loc[(testset.pred_label==0)].shape[0]
nr=testset.loc[(testset.pred_label==0)&(testset.target==0)].shape[0]/testset.loc[(testset.target==0)].shape[0]

print("N F1: %.2f"%(2*np*nr/(np+nr)))


positive::precision 0.400
positive::recall 0.833
negative::precision 0.760
negative::recall 0.297
F1: 0.54
N F1: 0.43


In [51]:
# another two runs. Only provide evaluations here.
pred=joblib.load('Data/pred.v5.1.4.round3') # load prediction result
assert pred.shape[0]==adj.shape[0]

pred_dict={}
for i,w in enumerate(adj['word'].tolist()):
    pred_dict[w]=pred[i]


testset['pred_score']=testset['word'].apply(lambda x: pred_dict[x])
testset['pred_label']=testset['pred_score'].apply(lambda x: 1 if x>0.5 else 0)

# positive::precision
print("positive::precision %.3f"%(testset.loc[(testset.pred_label==1)&(testset.target==1)].shape[0]/testset.loc[(testset.pred_label==1)].shape[0]))
print("positive::recall %.3f"%(testset.loc[(testset.pred_label==1)&(testset.target==1)].shape[0]/testset.loc[(testset.target==1)].shape[0]))


print("negative::precision %.3f"%(testset.loc[(testset.pred_label==0)&(testset.target==0)].shape[0]/testset.loc[(testset.pred_label==0)].shape[0]))
print("negative::recall %.3f"%(testset.loc[(testset.pred_label==0)&(testset.target==0)].shape[0]/testset.loc[(testset.target==0)].shape[0]))

pp=testset.loc[(testset.pred_label==1)&(testset.target==1)].shape[0]/testset.loc[(testset.pred_label==1)].shape[0]
pr=testset.loc[(testset.pred_label==1)&(testset.target==1)].shape[0]/testset.loc[(testset.target==1)].shape[0]

print("F1: %.2f"%(2*pp*pr/(pp+pr)))

np=testset.loc[(testset.pred_label==0)&(testset.target==0)].shape[0]/testset.loc[(testset.pred_label==0)].shape[0]
nr=testset.loc[(testset.pred_label==0)&(testset.target==0)].shape[0]/testset.loc[(testset.target==0)].shape[0]

print("N F1: %.2f"%(2*np*nr/(np+nr)))

positive::precision 0.558
positive::recall 0.667
negative::precision 0.789
negative::recall 0.703
F1: 0.61
N F1: 0.74


## Iter4

In [108]:
rs_train[:160].to_csv('Data/train.randomSample.r4.csv')

    Train model the same way as Section5.

In [52]:
pred=joblib.load('Data/pred.v5.2.round4') # load prediction result
assert pred.shape[0]==adj.shape[0]

pred_dict={}
for i,w in enumerate(adj['word'].tolist()):
    pred_dict[w]=pred[i]


testset['pred_score']=testset['word'].apply(lambda x: pred_dict[x])
testset['pred_label']=testset['pred_score'].apply(lambda x: 1 if x>0.5 else 0)

# positive::precision
print("positive::precision %.3f"%(testset.loc[(testset.pred_label==1)&(testset.target==1)].shape[0]/testset.loc[(testset.pred_label==1)].shape[0]))
print("positive::recall %.3f"%(testset.loc[(testset.pred_label==1)&(testset.target==1)].shape[0]/testset.loc[(testset.target==1)].shape[0]))


print("negative::precision %.3f"%(testset.loc[(testset.pred_label==0)&(testset.target==0)].shape[0]/testset.loc[(testset.pred_label==0)].shape[0]))
print("negative::recall %.3f"%(testset.loc[(testset.pred_label==0)&(testset.target==0)].shape[0]/testset.loc[(testset.target==0)].shape[0]))

pp=testset.loc[(testset.pred_label==1)&(testset.target==1)].shape[0]/testset.loc[(testset.pred_label==1)].shape[0]
pr=testset.loc[(testset.pred_label==1)&(testset.target==1)].shape[0]/testset.loc[(testset.target==1)].shape[0]

print("F1: %.2f"%(2*pp*pr/(pp+pr)))

np=testset.loc[(testset.pred_label==0)&(testset.target==0)].shape[0]/testset.loc[(testset.pred_label==0)].shape[0]
nr=testset.loc[(testset.pred_label==0)&(testset.target==0)].shape[0]/testset.loc[(testset.target==0)].shape[0]

print("N F1: %.2f"%(2*np*nr/(np+nr)))

positive::precision 0.750
positive::recall 0.583
negative::precision 0.792
negative::recall 0.891
F1: 0.66
N F1: 0.84


In [54]:
# another two runs. Only provide evaluations here.
pred=joblib.load('Data/pred.v5.22.round4') # load prediction result
assert pred.shape[0]==adj.shape[0]

pred_dict={}
for i,w in enumerate(adj['word'].tolist()):
    pred_dict[w]=pred[i]


testset['pred_score']=testset['word'].apply(lambda x: pred_dict[x])
testset['pred_label']=testset['pred_score'].apply(lambda x: 1 if x>0.5 else 0)

# positive::precision
print("positive::precision %.3f"%(testset.loc[(testset.pred_label==1)&(testset.target==1)].shape[0]/testset.loc[(testset.pred_label==1)].shape[0]))
print("positive::recall %.3f"%(testset.loc[(testset.pred_label==1)&(testset.target==1)].shape[0]/testset.loc[(testset.target==1)].shape[0]))


print("negative::precision %.3f"%(testset.loc[(testset.pred_label==0)&(testset.target==0)].shape[0]/testset.loc[(testset.pred_label==0)].shape[0]))
print("negative::recall %.3f"%(testset.loc[(testset.pred_label==0)&(testset.target==0)].shape[0]/testset.loc[(testset.target==0)].shape[0]))

pp=testset.loc[(testset.pred_label==1)&(testset.target==1)].shape[0]/testset.loc[(testset.pred_label==1)].shape[0]
pr=testset.loc[(testset.pred_label==1)&(testset.target==1)].shape[0]/testset.loc[(testset.target==1)].shape[0]

print("F1: %.2f"%(2*pp*pr/(pp+pr)))

np=testset.loc[(testset.pred_label==0)&(testset.target==0)].shape[0]/testset.loc[(testset.pred_label==0)].shape[0]
nr=testset.loc[(testset.pred_label==0)&(testset.target==0)].shape[0]/testset.loc[(testset.target==0)].shape[0]

print("N F1: %.2f"%(2*np*nr/(np+nr)))


positive::precision 0.724
positive::recall 0.583
negative::precision 0.789
negative::recall 0.875
F1: 0.65
N F1: 0.83


In [55]:
# another two runs. Only provide evaluations here.
pred=joblib.load('Data/pred.v5.2.4.round4') # load prediction result
assert pred.shape[0]==adj.shape[0]

pred_dict={}
for i,w in enumerate(adj['word'].tolist()):
    pred_dict[w]=pred[i]


testset['pred_score']=testset['word'].apply(lambda x: pred_dict[x])
testset['pred_label']=testset['pred_score'].apply(lambda x: 1 if x>0.5 else 0)

# positive::precision
print("positive::precision %.3f"%(testset.loc[(testset.pred_label==1)&(testset.target==1)].shape[0]/testset.loc[(testset.pred_label==1)].shape[0]))
print("positive::recall %.3f"%(testset.loc[(testset.pred_label==1)&(testset.target==1)].shape[0]/testset.loc[(testset.target==1)].shape[0]))


print("negative::precision %.3f"%(testset.loc[(testset.pred_label==0)&(testset.target==0)].shape[0]/testset.loc[(testset.pred_label==0)].shape[0]))
print("negative::recall %.3f"%(testset.loc[(testset.pred_label==0)&(testset.target==0)].shape[0]/testset.loc[(testset.target==0)].shape[0]))

pp=testset.loc[(testset.pred_label==1)&(testset.target==1)].shape[0]/testset.loc[(testset.pred_label==1)].shape[0]
pr=testset.loc[(testset.pred_label==1)&(testset.target==1)].shape[0]/testset.loc[(testset.target==1)].shape[0]

print("F1: %.2f"%(2*pp*pr/(pp+pr)))

np=testset.loc[(testset.pred_label==0)&(testset.target==0)].shape[0]/testset.loc[(testset.pred_label==0)].shape[0]
nr=testset.loc[(testset.pred_label==0)&(testset.target==0)].shape[0]/testset.loc[(testset.target==0)].shape[0]

print("N F1: %.2f"%(2*np*nr/(np+nr)))


positive::precision 0.690
positive::recall 0.556
negative::precision 0.775
negative::recall 0.859
F1: 0.62
N F1: 0.81


## Iter5

In [120]:
rs_train[:200].to_csv('Data/train.randomSample.r5.csv')

    Train model the same way as Section5.

In [56]:
pred=joblib.load('Data/pred.v5.3.round5') # load prediction result
assert pred.shape[0]==adj.shape[0]

pred_dict={}
for i,w in enumerate(adj['word'].tolist()):
    pred_dict[w]=pred[i]


testset['pred_score']=testset['word'].apply(lambda x: pred_dict[x])
testset['pred_label']=testset['pred_score'].apply(lambda x: 1 if x>0.5 else 0)

# positive::precision
print("positive::precision %.3f"%(testset.loc[(testset.pred_label==1)&(testset.target==1)].shape[0]/testset.loc[(testset.pred_label==1)].shape[0]))
print("positive::recall %.3f"%(testset.loc[(testset.pred_label==1)&(testset.target==1)].shape[0]/testset.loc[(testset.target==1)].shape[0]))


print("negative::precision %.3f"%(testset.loc[(testset.pred_label==0)&(testset.target==0)].shape[0]/testset.loc[(testset.pred_label==0)].shape[0]))
print("negative::recall %.3f"%(testset.loc[(testset.pred_label==0)&(testset.target==0)].shape[0]/testset.loc[(testset.target==0)].shape[0]))

pp=testset.loc[(testset.pred_label==1)&(testset.target==1)].shape[0]/testset.loc[(testset.pred_label==1)].shape[0]
pr=testset.loc[(testset.pred_label==1)&(testset.target==1)].shape[0]/testset.loc[(testset.target==1)].shape[0]

print("F1: %.2f"%(2*pp*pr/(pp+pr)))

np=testset.loc[(testset.pred_label==0)&(testset.target==0)].shape[0]/testset.loc[(testset.pred_label==0)].shape[0]
nr=testset.loc[(testset.pred_label==0)&(testset.target==0)].shape[0]/testset.loc[(testset.target==0)].shape[0]

print("N F1: %.2f"%(2*np*nr/(np+nr)))

positive::precision 0.792
positive::recall 0.528
negative::precision 0.776
negative::recall 0.922
F1: 0.63
N F1: 0.84


In [58]:
# another two runs. Only provide evaluations here.
pred=joblib.load('Data/pred.v5.33.round5') # load prediction result
assert pred.shape[0]==adj.shape[0]

pred_dict={}
for i,w in enumerate(adj['word'].tolist()):
    pred_dict[w]=pred[i]


testset['pred_score']=testset['word'].apply(lambda x: pred_dict[x])
testset['pred_label']=testset['pred_score'].apply(lambda x: 1 if x>0.5 else 0)

# positive::precision
print("positive::precision %.3f"%(testset.loc[(testset.pred_label==1)&(testset.target==1)].shape[0]/testset.loc[(testset.pred_label==1)].shape[0]))
print("positive::recall %.3f"%(testset.loc[(testset.pred_label==1)&(testset.target==1)].shape[0]/testset.loc[(testset.target==1)].shape[0]))


print("negative::precision %.3f"%(testset.loc[(testset.pred_label==0)&(testset.target==0)].shape[0]/testset.loc[(testset.pred_label==0)].shape[0]))
print("negative::recall %.3f"%(testset.loc[(testset.pred_label==0)&(testset.target==0)].shape[0]/testset.loc[(testset.target==0)].shape[0]))

pp=testset.loc[(testset.pred_label==1)&(testset.target==1)].shape[0]/testset.loc[(testset.pred_label==1)].shape[0]
pr=testset.loc[(testset.pred_label==1)&(testset.target==1)].shape[0]/testset.loc[(testset.target==1)].shape[0]

print("F1: %.2f"%(2*pp*pr/(pp+pr)))

np=testset.loc[(testset.pred_label==0)&(testset.target==0)].shape[0]/testset.loc[(testset.pred_label==0)].shape[0]
nr=testset.loc[(testset.pred_label==0)&(testset.target==0)].shape[0]/testset.loc[(testset.target==0)].shape[0]

print("N F1: %.2f"%(2*np*nr/(np+nr)))


positive::precision 0.667
positive::recall 0.611
negative::precision 0.791
negative::recall 0.828
F1: 0.64
N F1: 0.81


In [59]:
# another two runs. Only provide evaluations here.
pred=joblib.load('Data/pred.v5.32.round5') # load prediction result
assert pred.shape[0]==adj.shape[0]

pred_dict={}
for i,w in enumerate(adj['word'].tolist()):
    pred_dict[w]=pred[i]


testset['pred_score']=testset['word'].apply(lambda x: pred_dict[x])
testset['pred_label']=testset['pred_score'].apply(lambda x: 1 if x>0.5 else 0)

# positive::precision
print("positive::precision %.3f"%(testset.loc[(testset.pred_label==1)&(testset.target==1)].shape[0]/testset.loc[(testset.pred_label==1)].shape[0]))
print("positive::recall %.3f"%(testset.loc[(testset.pred_label==1)&(testset.target==1)].shape[0]/testset.loc[(testset.target==1)].shape[0]))


print("negative::precision %.3f"%(testset.loc[(testset.pred_label==0)&(testset.target==0)].shape[0]/testset.loc[(testset.pred_label==0)].shape[0]))
print("negative::recall %.3f"%(testset.loc[(testset.pred_label==0)&(testset.target==0)].shape[0]/testset.loc[(testset.target==0)].shape[0]))

pp=testset.loc[(testset.pred_label==1)&(testset.target==1)].shape[0]/testset.loc[(testset.pred_label==1)].shape[0]
pr=testset.loc[(testset.pred_label==1)&(testset.target==1)].shape[0]/testset.loc[(testset.target==1)].shape[0]

print("F1: %.2f"%(2*pp*pr/(pp+pr)))

np=testset.loc[(testset.pred_label==0)&(testset.target==0)].shape[0]/testset.loc[(testset.pred_label==0)].shape[0]
nr=testset.loc[(testset.pred_label==0)&(testset.target==0)].shape[0]/testset.loc[(testset.target==0)].shape[0]

print("N F1: %.2f"%(2*np*nr/(np+nr)))


positive::precision 0.610
positive::recall 0.694
negative::precision 0.814
negative::recall 0.750
F1: 0.65
N F1: 0.78


    Annotate 80~100w/iter. Worse performance than HighEntropy.

# Release

In [77]:
pred=joblib.load('Data/pred.v6.4.3.round4') # load prediction result. We use this version of prediction
# as its precision and recall are more balanced.
assert pred.shape[0]==adj.shape[0]

pred_dict={}
for i,w in enumerate(adj['word'].tolist()):
    pred_dict[w]=pred[i]

In [78]:
# >0.5, label as "mental", otherwise "physical"

results={}
for k,v in pred_dict.items():
    results[k]='mental' if v>0.5 else 'physical'

In [81]:
with open("Data/infer_mental_adj.json", "w") as outfile:
    json.dump(results, outfile)

# load
"""
with open("Data/infer_sub_adj.json", "r") as infile:
    x=json.load(infile)
"""

In [82]:
# mental ratio
pos_cnt=0
for w,c in results.items():
    if c=='mental':
        pos_cnt+=1
print(pos_cnt/len(results))

0.3272078990674712


# Compare with SentiWordNet

## Process

In [60]:
sentinet=pd.read_csv('Data/SentiWordNet3.csv')

In [61]:
sentinet.head()

,Unnamed: 0,POS,ID,PosScore,NegScore,SynsetTerms,Gloss
0,0,a,1740.0,0.125,0.00,able#1,(usually followed by `to') having the necessar...
1,1,a,2098.0,0.000,0.75,unable#1,(usually followed by `to') not having the nece...
2,2,a,2312.0,0.000,0.00,dorsal#2 abaxial#1,facing away from the axis of an organ or organ...
3,3,a,2527.0,0.000,0.00,ventral#2 adaxial#1,nearest to or facing toward the axis of an org...
4,4,a,2730.0,0.000,0.00,acroscopic#1,facing or on the side toward the apex


In [62]:
sentinet.shape

(117660, 7)

In [63]:
sentinet['ObjScore']=1-(sentinet['PosScore']+sentinet['NegScore'])

SentiClass=[]
for o,p,n in zip(sentinet['ObjScore'], sentinet['PosScore'], sentinet['NegScore']):
    if max(o,p,n)==o:
        SentiClass.append('obj')
        continue
    if max(o,p,n)==p:
        SentiClass.append('pos')
        continue
    if max(o,p,n)==n:
        SentiClass.append('neg')
        continue
    else:
        SentiClass.append('unknown')

sentinet['SentiClass']=SentiClass

In [65]:
# get positive terms from sentinet
senti_pos_set=set()
for s in sentinet.loc[(sentinet.POS=='a')&(sentinet.SentiClass=='pos'),'SynsetTerms']:
    for w in s.split(' '):
        _w=w.split('#')[0]
        senti_pos_set.add(_w)

In [67]:
len(senti_pos_set)

2098

In [66]:
# get negative terms from sentinet
senti_neg_set=set()
for s in sentinet.loc[(sentinet.POS=='a')&(sentinet.SentiClass=='neg'),'SynsetTerms']:
    for w in s.split(' '):
        _w=w.split('#')[0]
        senti_neg_set.add(_w)

In [68]:
len(senti_neg_set)

3241

In [64]:
# get objective terms from sentinet
senti_obj_set=set()
for s in sentinet.loc[(sentinet.POS=='a')&(sentinet.SentiClass=='obj'),'SynsetTerms']:
    for w in s.split(' '):
        _w=w.split('#')[0]
        senti_obj_set.add(_w)

In [69]:
len(senti_obj_set)

18241

In [70]:
senti_words=senti_obj_set.union(senti_pos_set.union(senti_neg_set))

In [71]:
len(senti_words)

21479

## GetWordSentiment

In [ ]:
# If a word only appears in `senti_pos_set` or `senti_neg_set`, it is regarded as `Subjective`. 
# If a word only appears in `senti_obj_set`, it is regarded as `Objective`. 
# If a word appears both in `senti_obj_set` and union(`senti_pos_set`, `senti_neg_set`), 
# it is regarded as `Dual`, which means this word could bear subjective or objective meanings under different
# contexts.

In [72]:
pure_senti_sub_set=(senti_pos_set.union(senti_neg_set)).difference(senti_obj_set)

pure_senti_obj_set=senti_obj_set.difference(senti_pos_set.union(senti_neg_set))

dual_senti_set=(senti_pos_set.union(senti_neg_set)).intersection(senti_obj_set)

In [74]:
print("Total adjectives in sentinet: %d"%(len(pure_senti_sub_set)+len(pure_senti_obj_set)+len(dual_senti_set)))

print("Subjective adjective ratio: %.2f%%"%(100*len(pure_senti_sub_set)/(len(pure_senti_sub_set)+len(pure_senti_obj_set)+len(dual_senti_set))))

print("Objective adjective ratio: %.2f%%"%(100*len(pure_senti_obj_set)/(len(pure_senti_sub_set)+len(pure_senti_obj_set)+len(dual_senti_set))))

print("Dual adjective ratio: %.2f%%"%(100*len(dual_senti_set)/(len(pure_senti_sub_set)+len(pure_senti_obj_set)+len(dual_senti_set))))

Total adjectives in sentinet: 21479
Subjective adjective ratio: 15.08%
Objective adjective ratio: 76.38%
Dual adjective ratio: 8.54%


## Difference

In [ ]:
# distribution of Subjective, Objective and Dual adjectives in Mental and Physical classes

In [84]:
mental_set=set()
for w,c in results.items():
    if c=='mental':
        mental_set.add(w)
        
physical_set=set()
for w,c in results.items():
    if c=='physical':
        physical_set.add(w)

In [87]:
mental_obj=mental_set.intersection(pure_senti_obj_set)
mental_sub=mental_set.intersection(pure_senti_sub_set)
mental_dual=mental_set.intersection(dual_senti_set)

# process miss words due to comparative format
mental_miss=set()
for w in mental_set:
    if w in mental_obj or w in mental_sub or w in mental_dual:
        pass
    else:
        mental_miss.add(w)

mental_obj_c=set()
mental_sub_c=set()
mental_dual_c=set()

# get word prototype
lemmatizer = WordNetLemmatizer()

for w in mental_miss:
    _w=lemmatizer.lemmatize(w,pos='a')
    if _w in pure_senti_obj_set:
        mental_obj_c.add(w)
    elif _w in pure_senti_sub_set:
        mental_sub_c.add(w)
    elif _w in dual_senti_set:
        mental_dual_c.add(w)
    else:
        print(w)
        
print("Subjective in Mental adjs: %.2f%%"%(100*len(mental_sub.union(mental_sub_c))/(len(mental_set))))

print("Obj in Mental adjs: %.2f%%"%(100*len(mental_obj.union(mental_obj_c))/(len(mental_set))))

print("Dual polarity in Mental adjs: %.2f%%"%(100*len(mental_dual.union(mental_dual_c))/(len(mental_set))))

Subjective in Mental adjs: 28.08%
Obj in Mental adjs: 43.00%
Dual polarity in Mental adjs: 28.92%


In [89]:
phy_obj=physical_set.intersection(pure_senti_obj_set)
phy_sub=physical_set.intersection(pure_senti_sub_set)
phy_dual=physical_set.intersection(dual_senti_set)

phy_miss=set()
for w in physical_set:
    if w in phy_obj or w in phy_sub or w in phy_dual:
        pass
    else:
        phy_miss.add(w)
print(len(phy_miss))

# process comparatives
phy_obj_c=set()
phy_sub_c=set()
phy_dual_c=set()

# get word prototype
lemmatizer = WordNetLemmatizer()

for w in phy_miss:
    if (not isinstance(w,str)) and numpy.isnan(w):
        continue
    _w=lemmatizer.lemmatize(w,pos='a')
    if _w in pure_senti_obj_set:
        phy_obj_c.add(w)
    elif _w in pure_senti_sub_set:
        phy_sub_c.add(w)
    elif _w in dual_senti_set:
        phy_dual_c.add(w)
    else:
        print(w)
        
print("Subjective in Physical adjs: %.2f%%"%(100*len(phy_sub.union(phy_sub_c))/(len(physical_set))))

print("Obj in Physical adjs: %.2f%%"%(100*len(phy_obj.union(phy_obj_c))/(len(physical_set))))

print("Dual polarity in Physical adjs: %.2f%%"%(100*len(phy_dual.union(phy_dual_c))/(len(physical_set))))


314
Subjective in Physical adjs: 9.13%
Obj in Physical adjs: 74.40%
Dual polarity in Physical adjs: 16.45%


    From the above distributions, we find that 43% of the Mental adjs are labeled as objectives in sentiwordnet. This indicates the definition of Mental/Physical is different from Subjective/Objective in sentiment analysis task. In fact, many objective words bear mental functionalities. We show word examples below in six categories: emotion, motive, perceiving, reasoning, planning and personality. 

In [ ]:
# get mental-objective adj
for w in mental_obj:
    print(w)

    Emotion: favored, scorned, trustworthy, frisky, gripping, stilted
    
    Need: devout, deserving, hired, protective, wealthy, rewarding
    
    Perceive: sensuous, ubiquitous, instinctive, detected, recognized, perceivable
    
    Reason: suitable, predominate, substandard, critical, relevant, causal
    
    Plan: committed, aimless, exploited, unplanned, purposeful, executed
  
    Personality: whimsical, squeamish, entrepreneurial, punctual, shy, intrepid
    